# Exploración de Datos (EDA) - SynthRAD Dataset

Este notebook realiza una exploración completa del dataset médico SynthRAD, incluyendo:
- Carga eficiente de datos usando MONAI CacheDataset
- Visualización de imágenes MR, CT y máscaras
- Análisis del spacing (resolución espacial) de cada volumen
- Estadísticas básicas de intensidad
- Análisis de dimensiones de volúmenes

**Nota**: Este notebook está optimizado para ordenadores con recursos limitados usando batch_size=1 y cache dataset.

## 1. Import Required Libraries

Importamos todas las librerías necesarias para el análisis de datos médicos.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# MONAI imports
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd,
    Orientationd, Spacingd, ScaleIntensityRanged, ToTensord
)
from monai.utils import set_determinism

# Local imports
from constants import TASK1_HN, TASK1_AB, TASK1_TH, TASK2_HN, TASK2_AB, TASK2_TH
from utils.utils import get_data_paths

# Set random seed for reproducibility
set_determinism(42)

print("Librerías importadas correctamente")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

Librerías importadas correctamente
NumPy version: 1.26.4
Matplotlib version: 3.10.1


## 2. Configuración del Dataset

Configuraremos el dataset que queremos explorar. Puedes cambiar la variable `DATASET_PATH` para explorar diferentes anatomías.

In [2]:
# Configuración del dataset a explorar
# Cambia esta variable para explorar diferentes anatomías:
# TASK1_HN: Head & Neck (Cabeza y Cuello)
# TASK1_AB: Abdomen
# TASK1_TH: Thorax (Tórax)
# También puedes usar TASK2_* para el Task 2

DATASET_PATH = TASK1_TH  # Cambia aquí para explorar diferentes datasets
DATASET_NAME = "TASK1_TH"

print(f"Dataset seleccionado: {DATASET_NAME}")
print(f"Ruta: {DATASET_PATH}")

# Verificar que existe el dataset
if isinstance(DATASET_PATH, list):
    for path in DATASET_PATH:
        if not path.exists():
            print(f"Advertencia: La ruta {path} no existe")
        else:
            print(f"Ruta encontrada: {path}")
else:
    if not DATASET_PATH.exists():
        print(f"Advertencia: La ruta {DATASET_PATH} no existe")
    else:
        print(f"Ruta encontrada: {DATASET_PATH}")

Dataset seleccionado: TASK1_TH
Ruta: data\synthRAD2025_Task1_Train\Task1\TH
Ruta encontrada: data\synthRAD2025_Task1_Train\Task1\TH


## 3. Carga de Datos con MONAI CacheDataset

Utilizamos MONAI CacheDataset para una carga eficiente en sistemas con recursos limitados.

In [3]:
# Obtener rutas de archivos
print("Buscando archivos...")
image_paths, ct_paths, mask_paths = get_data_paths(DATASET_PATH, debug=False)

print(f"Archivos encontrados:")
print(f"   - Imágenes MR: {len(image_paths)}")
print(f"   - Imágenes CT: {len(ct_paths)}")
print(f"   - Máscaras: {len(mask_paths)}")

# Crear lista de diccionarios para el dataset
data_dicts = []
for i, (img_path, ct_path, mask_path) in enumerate(zip(image_paths, ct_paths, mask_paths)):
    data_dicts.append({
        "image": img_path,      # MR image
        "label": ct_path,       # CT image (target)
        "mask": mask_path,      # Segmentation mask
    })

print(f"\nPrimer ejemplo:")
for key, value in data_dicts[0].items():
    print(f"   {key}: {value}")

Buscando archivos...
Archivos encontrados:
   - Imágenes MR: 182
   - Imágenes CT: 182
   - Máscaras: 182

Primer ejemplo:
   image: data\synthRAD2025_Task1_Train\Task1\TH\1THA001\mr.mha
   label: data\synthRAD2025_Task1_Train\Task1\TH\1THA001\ct.mha
   mask: data\synthRAD2025_Task1_Train\Task1\TH\1THA001\mask.mha


In [4]:
# Definir transformaciones básicas para EDA
eda_transforms = Compose([
    LoadImaged(keys=["image", "label", "mask"], image_only=False, ensure_channel_first=True, reader="NibabelReader"),
    Orientationd(keys=["image", "label", "mask"], axcodes="RAS"),
])

print("Transformaciones definidas:")
print("   - LoadImaged: Carga archivos .mha")
print("   - EnsureChannelFirstd: Formato [C, H, W, D]")

Transformaciones definidas:
   - LoadImaged: Carga archivos .mha
   - EnsureChannelFirstd: Formato [C, H, W, D]


In [5]:
# Crear CacheDataset para carga eficiente
print("Creando CacheDataset...")
eda_dataset = CacheDataset(
    data=data_dicts,
    transform=eda_transforms,
    cache_rate=1.0,  # Cache al 100% para mejor rendimiento en datasets pequeños
    num_workers=1   # Un solo worker para sistemas con pocos recursos
)

# Crear DataLoader con batch_size=1
eda_loader = DataLoader(
    eda_dataset,
    batch_size=1,
    shuffle=False,  # No mezclamos para análisis ordenado
    num_workers=0   # Sin workers adicionales para sistemas con pocos recursos
)

print(f"Dataset creado exitosamente")
print(f"   - Número de muestras: {len(eda_dataset)}")
print(f"   - Batch size: 1")
print(f"   - Cache rate: 100%")

Creando CacheDataset...


Loading dataset: 100%|██████████| 182/182 [02:28<00:00,  1.23it/s]

Dataset creado exitosamente
   - Número de muestras: 182
   - Batch size: 1
   - Cache rate: 100%


In [6]:
# Mostramos las keys del metadata del primer batch
batch = next(iter(eda_loader))
batch_keys = batch['image_meta_dict'].keys()
print(f"\nKeys del primer batch: {list(batch_keys)}")

for key in batch_keys:
    print(f"   {key}: {batch['image_meta_dict'][key]}")



Keys del primer batch: ['Modality', 'spacing', original_affine, space, affine, spatial_shape, original_channel_dim, 'filename_or_obj']
   Modality: ['MET_MOD_UNKNOWN']
   spacing: tensor([[1., 1., 3.]], dtype=torch.float64)
   original_affine: tensor([[[  -1.0000,    0.0000,    0.0000,  254.5000],
         [   0.0000,   -1.0000,    0.0000,   91.5000],
         [   0.0000,    0.0000,    3.0000, -134.9999],
         [   0.0000,    0.0000,    0.0000,    1.0000]]], dtype=torch.float64)
   space: [RAS]
   affine: tensor([[[  -1.0000,    0.0000,    0.0000,  254.5000],
         [   0.0000,   -1.0000,    0.0000,   91.5000],
         [   0.0000,    0.0000,    3.0000, -134.9999],
         [   0.0000,    0.0000,    0.0000,    1.0000]]], dtype=torch.float64)
   spatial_shape: tensor([[499, 313, 103]], dtype=torch.int32)
   original_channel_dim: tensor([nan], dtype=torch.float64)
   filename_or_obj: ['data\\synthRAD2025_Task1_Train\\Task1\\TH\\1THA001\\mr.mha']


In [7]:
# Vamos a crear un bucle que dumpee para cada volumen los metadatos de las imágenes. Tiene que ser un csv para todos los volúmenes.
# Vamos a crear una carpeta metadata bajo /data/ para guardar los CSVs de metadatos. El nombre del csv será metadata_{task}_{anatomy}.csv

metadata_dir = Path("data/metadata")
metadata_dir.mkdir(parents=True, exist_ok=True)

metadata_file = metadata_dir / f"metadata_{DATASET_NAME.replace(' ', '_').lower()}.csv"
print(f"\nGuardando metadatos en: {metadata_file}")

# Creamos un DataFrame para almacenar los metadatos
metadata_df = pd.DataFrame()

# Iteramos sobre el DataLoader y extraemos los metadatos
for i, batch in enumerate(eda_loader):
    # Extraemos los metadatos de la imagen
    meta = batch['image_meta_dict']

    # Creamos un diccionario con los metadatos relevantes
    meta_dict = {
        'index': i,
        'Modality': meta.get('Modality', 'N/A'),
        'spacing': meta.get('spacing', 'N/A'),
        'original_affine': meta.get('original_affine', 'N/A'),
        'space': meta.get('space', 'N/A'),
        'affine': meta.get('affine', 'N/A'),
        'spatial_shape': meta.get('spatial_shape', 'N/A'),
        'origintal_channel_dim': meta.get('original_channel_dim', 'N/A'),
        'filename_or_obj': meta.get('filename_or_obj', 'N/A'),
        'image_shape': batch['image'].shape if 'image' in batch else 'N/A',
    }

    # Añadimos al DataFrame
    metadata_df = pd.concat([metadata_df, pd.DataFrame([meta_dict])], ignore_index=True)

# Guardamos el DataFrame como CSV
metadata_df.to_csv(metadata_file, index=False)



Guardando metadatos en: data\metadata\metadata_task1_th.csv


## 4. Análisis de Spacing (Resolución Espacial)

El spacing indica la resolución espacial de cada vóxel en mm. Es crucial para entender la calidad y resolución de las imágenes médicas.

In [ ]:
def extract_spacing_info(dataset_loader):
    """Extrae información de spacing de todas las muestras del dataset"""
    spacing_data = []

    print("Extrayendo información de spacing...")

    for i, batch in enumerate(dataset_loader):
        # Los metadatos están en las claves con sufijo '_meta_dict'
        image_meta = batch['image_meta_dict']
        label_meta = batch['label_meta_dict']
        mask_meta = batch['mask_meta_dict']
        print(f"Metadata keys: {image_meta.keys()}")
        # Extraer spacing (resolución espacial)

        img_spacing = image_meta['affine'][0][1:4].numpy()  # [x, y, z] spacing
        label_spacing = label_meta['affine'][0][1:4].numpy()
        mask_spacing = mask_meta['affine'][0][1:4].numpy()

        # Extraer dimensiones originales
        img_shape = image_meta['spatial_shape'][0].numpy()
        label_shape = label_meta['spatial_shape'][0].numpy()
        mask_shape = mask_meta['spatial_shape'][0].numpy()

        spacing_data.append({
            'sample_id': i,
            'image_spacing_x': img_spacing[0],
            'image_spacing_y': img_spacing[1],
            'image_spacing_z': img_spacing[2],
            'image_shape_x': img_shape[0],
            'image_shape_y': img_shape[1],
            'image_shape_z': img_shape[2],
            'label_spacing_x': label_spacing[0],
            'label_spacing_y': label_spacing[1],
            'label_spacing_z': label_spacing[2],
            'mask_spacing_x': mask_spacing[0],
            'mask_spacing_y': mask_spacing[1],
            'mask_spacing_z': mask_spacing[2],
        })

        print(f"\nMuestra {i}:")
        print(f"   MR Image - Spacing: [{str(img_spacing[0])}, {str(img_spacing[1])}, {str(img_spacing[2])}] mm")
        print(f"   CT Label - Spacing: [{str(label_spacing[0])}, {str(label_spacing[1])}, {str(label_spacing[2])}] mm")
        print(f"   Mask - Spacing: [{str(mask_spacing[0])}, {str(mask_spacing[1])}, {str(mask_spacing[2])}] mm")


        print(f"   Volume físico: [{int(img_shape[0])*img_spacing[0]:.1f}, {int(img_shape[1])*img_spacing[1]:.1f}, {int(img_shape[2])*img_spacing[2]:.1f}] mm")

    return pd.DataFrame(spacing_data)

# Extraer información de spacing
spacing_df = extract_spacing_info(eda_loader)
print(f"\nInformación de spacing extraída para {len(spacing_df)} muestras")

Extrayendo información de spacing...
Metadata keys: dict_keys(['Modality', 'spacing', original_affine, space, affine, spatial_shape, original_channel_dim, 'filename_or_obj'])

Muestra 0:
   MR Image - Spacing: [[  0.          -1.           0.         396.01171875], [  0.   0.   3. -99.], [0. 0. 0. 1.]] mm
   CT Label - Spacing: [[  0.          -1.           0.         396.01171875], [  0.   0.   3. -99.], [0. 0. 0. 1.]] mm
   Mask - Spacing: [[  0.          -1.           0.         396.01171875], [  0.   0.   3. -99.], [0. 0. 0. 1.]] mm


Extrayendo información de spacing...
Metadata keys: dict_keys(['Modality', 'spacing', original_affine, space, affine, spatial_shape, original_channel_dim, 'filename_or_obj'])

Muestra 0:
   MR Image - Spacing: [[  0.          -1.           0.         396.01171875], [  0.   0.   3. -99.], [0. 0. 0. 1.]] mm
   CT Label - Spacing: [[  0.          -1.           0.         396.01171875], [  0.   0.   3. -99.], [0. 0. 0. 1.]] mm
   Mask - Spacing: [[  0.          -1.           0.         396.01171875], [  0.   0.   3. -99.], [0. 0. 0. 1.]] mm


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [ ]:
# Mostrar estadísticas de spacing
print("ESTADÍSTICAS DE SPACING (resolución espacial en mm):")
print("="*60)

spacing_stats = spacing_df[['image_spacing_x', 'image_spacing_y', 'image_spacing_z']].describe()
print("\nMR Images:")
print(spacing_stats.round(4))

# Verificar consistencia entre MR y CT
print("\nVerificación de consistencia MR vs CT:")
spacing_diff_x = (spacing_df['image_spacing_x'] - spacing_df['label_spacing_x']).abs()
spacing_diff_y = (spacing_df['image_spacing_y'] - spacing_df['label_spacing_y']).abs()
spacing_diff_z = (spacing_df['image_spacing_z'] - spacing_df['label_spacing_z']).abs()

print(f"   Diferencia máxima en X: {spacing_diff_x.max():.6f} mm")
print(f"   Diferencia máxima en Y: {spacing_diff_y.max():.6f} mm")
print(f"   Diferencia máxima en Z: {spacing_diff_z.max():.6f} mm")

if spacing_diff_x.max() < 0.001 and spacing_diff_y.max() < 0.001 and spacing_diff_z.max() < 0.001:
    print("   MR e imágenes CT tienen spacing consistente")
else:
    print("   Hay diferencias de spacing entre MR y CT")

# Mostrar rango de volúmenes físicos
volumes_x = spacing_df['image_shape_x'] * spacing_df['image_spacing_x']
volumes_y = spacing_df['image_shape_y'] * spacing_df['image_spacing_y']
volumes_z = spacing_df['image_shape_z'] * spacing_df['image_spacing_z']

print(f"\nDIMENSIONES FÍSICAS (en mm):")
print(f"   Rango X: {volumes_x.min():.1f} - {volumes_x.max():.1f} mm")
print(f"   Rango Y: {volumes_y.min():.1f} - {volumes_y.max():.1f} mm")
print(f"   Rango Z: {volumes_z.min():.1f} - {volumes_z.max():.1f} mm")

## 5. Visualización de Spacing y Dimensiones

Creamos gráficos para visualizar la distribución de spacing y dimensiones en el dataset.

In [ ]:
# Crear visualizaciones de spacing
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle(f'Análisis de Spacing y Dimensiones - {DATASET_NAME}', fontsize=16, fontweight='bold')

# Primera fila: Distribución de spacing
spacing_data = spacing_df[['image_spacing_x', 'image_spacing_y', 'image_spacing_z']]

axes[0, 0].hist(spacing_data['image_spacing_x'], bins=20, alpha=0.7, color='red', edgecolor='black')
axes[0, 0].set_title('Spacing X (mm)')
axes[0, 0].set_xlabel('Resolución (mm)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(spacing_data['image_spacing_y'], bins=20, alpha=0.7, color='green', edgecolor='black')
axes[0, 1].set_title('Spacing Y (mm)')
axes[0, 1].set_xlabel('Resolución (mm)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].hist(spacing_data['image_spacing_z'], bins=20, alpha=0.7, color='blue', edgecolor='black')
axes[0, 2].set_title('Spacing Z (mm)')
axes[0, 2].set_xlabel('Resolución (mm)')
axes[0, 2].set_ylabel('Frecuencia')
axes[0, 2].grid(True, alpha=0.3)

# Segunda fila: Distribución de dimensiones
shape_data = spacing_df[['image_shape_x', 'image_shape_y', 'image_shape_z']]

axes[1, 0].hist(shape_data['image_shape_x'], bins=20, alpha=0.7, color='red', edgecolor='black')
axes[1, 0].set_title('Dimensiones X (voxels)')
axes[1, 0].set_xlabel('Número de voxels')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(shape_data['image_shape_y'], bins=20, alpha=0.7, color='green', edgecolor='black')
axes[1, 1].set_title('Dimensiones Y (voxels)')
axes[1, 1].set_xlabel('Número de voxels')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].hist(shape_data['image_shape_z'], bins=20, alpha=0.7, color='blue', edgecolor='black')
axes[1, 2].set_title('Dimensiones Z (voxels)')
axes[1, 2].set_xlabel('Número de voxels')
axes[1, 2].set_ylabel('Frecuencia')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabla resumen
print("\nTABLA RESUMEN DE SPACING:")
print(spacing_df[['subject_id', 'image_spacing_x', 'image_spacing_y', 'image_spacing_z',
                  'image_shape_x', 'image_shape_y', 'image_shape_z']].head(10))

## 6. Visualización de Imágenes Médicas

Visualizamos muestras de imágenes MR, CT y máscaras en diferentes orientaciones anatómicas.

In [ ]:
def plot_medical_slices(image, label, mask, title_prefix="Sample", slice_ratios=[0.3, 0.5, 0.7]):
    """Plotea slices de imágenes médicas en diferentes orientaciones"""

    # Remover dimensión de batch y canal
    img_vol = image[0, 0].numpy()  # [H, W, D]
    label_vol = label[0, 0].numpy()
    mask_vol = mask[0, 0].numpy()

    fig, axes = plt.subplots(3, 9, figsize=(20, 8))
    fig.suptitle(f'{title_prefix} - Orientaciones Anatómicas', fontsize=16, fontweight='bold')

    # Para cada slice ratio
    for col, ratio in enumerate(slice_ratios):
        # Axial (vista superior/inferior)
        axial_idx = int(img_vol.shape[2] * ratio)
        axes[0, col*3].imshow(img_vol[:, :, axial_idx], cmap='gray')
        axes[0, col*3].set_title(f'MR Axial (z={axial_idx})')
        axes[0, col*3].axis('off')

        axes[0, col*3+1].imshow(label_vol[:, :, axial_idx], cmap='gray')
        axes[0, col*3+1].set_title(f'CT Axial (z={axial_idx})')
        axes[0, col*3+1].axis('off')

        axes[0, col*3+2].imshow(mask_vol[:, :, axial_idx], cmap='viridis')
        axes[0, col*3+2].set_title(f'Mask Axial (z={axial_idx})')
        axes[0, col*3+2].axis('off')

        # Sagital (vista lateral)
        sagital_idx = int(img_vol.shape[0] * ratio)
        axes[1, col*3].imshow(img_vol[sagital_idx, :, :], cmap='gray')
        axes[1, col*3].set_title(f'MR Sagital (x={sagital_idx})')
        axes[1, col*3].axis('off')

        axes[1, col*3+1].imshow(label_vol[sagital_idx, :, :], cmap='gray')
        axes[1, col*3+1].set_title(f'CT Sagital (x={sagital_idx})')
        axes[1, col*3+1].axis('off')

        axes[1, col*3+2].imshow(mask_vol[sagital_idx, :, :], cmap='viridis')
        axes[1, col*3+2].set_title(f'Mask Sagital (x={sagital_idx})')
        axes[1, col*3+2].axis('off')

        # Coronal (vista frontal/posterior)
        coronal_idx = int(img_vol.shape[1] * ratio)
        axes[2, col*3].imshow(img_vol[:, coronal_idx, :], cmap='gray')
        axes[2, col*3].set_title(f'MR Coronal (y={coronal_idx})')
        axes[2, col*3].axis('off')

        axes[2, col*3+1].imshow(label_vol[:, coronal_idx, :], cmap='gray')
        axes[2, col*3+1].set_title(f'CT Coronal (y={coronal_idx})')
        axes[2, col*3+1].axis('off')

        axes[2, col*3+2].imshow(mask_vol[:, coronal_idx, :], cmap='viridis')
        axes[2, col*3+2].set_title(f'Mask Coronal (y={coronal_idx})')
        axes[2, col*3+2].axis('off')

    plt.tight_layout()
    plt.show()

    # Mostrar estadísticas de intensidad
    print(f"\nEstadísticas de intensidad - {title_prefix}:")
    print(f"   MR Image: min={img_vol.min():.2f}, max={img_vol.max():.2f}, mean={img_vol.mean():.2f}, std={img_vol.std():.2f}")
    print(f"   CT Label: min={label_vol.min():.2f}, max={label_vol.max():.2f}, mean={label_vol.mean():.2f}, std={label_vol.std():.2f}")
    print(f"   Mask: min={mask_vol.min():.2f}, max={mask_vol.max():.2f}, unique values={len(np.unique(mask_vol))}")
    print(f"   Shape: MR={img_vol.shape}, CT={label_vol.shape}, Mask={mask_vol.shape}")

print("Función de visualización definida")

In [ ]:
# Visualizar las primeras muestras del dataset
print("Visualizando muestras del dataset...")

# Obtener las primeras muestras
samples_to_show = min(2, len(eda_dataset))  # Mostrar máximo 2 muestras para no sobrecargar

for i in range(samples_to_show):
    print(f"\n{'='*50}")
    print(f"MUESTRA {i+1}/{samples_to_show}")
    print(f"{'='*50}")

    # Obtener la muestra
    sample = eda_dataset[i]

    # Extraer información de la muestra
    image = sample['image'].unsqueeze(0)  # Añadir dimensión de batch
    label = sample['label'].unsqueeze(0)
    mask = sample['mask'].unsqueeze(0)
    subject_id = sample['subject_id']

    print(f"Subject ID: {subject_id}")

    # Mostrar información de spacing de esta muestra específica
    sample_spacing = spacing_df.iloc[i]
    print(f"Spacing: [{sample_spacing['image_spacing_x']:.3f}, {sample_spacing['image_spacing_y']:.3f}, {sample_spacing['image_spacing_z']:.3f}] mm")
    print(f"Dimensiones: [{sample_spacing['image_shape_x']}, {sample_spacing['image_shape_y']}, {sample_spacing['image_shape_z']}] voxels")

    # Visualizar
    plot_medical_slices(image, label, mask, f"Muestra {i+1} ({subject_id})")

print(f"\nVisualización completada para {samples_to_show} muestras")

## 7. Análisis Estadístico Detallado

Analizamos las estadísticas de intensidad y características del dataset completo.

In [ ]:
def analyze_intensity_statistics(dataset_loader):
    """Analiza estadísticas de intensidad de todo el dataset"""

    print("Analizando estadísticas de intensidad del dataset completo...")

    # Listas para acumular estadísticas
    mr_stats = {'min': [], 'max': [], 'mean': [], 'std': []}
    ct_stats = {'min': [], 'max': [], 'mean': [], 'std': []}
    mask_stats = {'unique_values': [], 'non_zero_ratio': []}

    for i, batch in enumerate(dataset_loader):
        # Extraer volúmenes (sin dimensión de batch y canal)
        mr_vol = batch['image'][0, 0].numpy()
        ct_vol = batch['label'][0, 0].numpy()
        mask_vol = batch['mask'][0, 0].numpy()

        # Estadísticas MR
        mr_stats['min'].append(mr_vol.min())
        mr_stats['max'].append(mr_vol.max())
        mr_stats['mean'].append(mr_vol.mean())
        mr_stats['std'].append(mr_vol.std())

        # Estadísticas CT
        ct_stats['min'].append(ct_vol.min())
        ct_stats['max'].append(ct_vol.max())
        ct_stats['mean'].append(ct_vol.mean())
        ct_stats['std'].append(ct_vol.std())

        # Estadísticas Mask
        unique_vals = len(np.unique(mask_vol))
        non_zero_ratio = np.count_nonzero(mask_vol) / mask_vol.size
        mask_stats['unique_values'].append(unique_vals)
        mask_stats['non_zero_ratio'].append(non_zero_ratio)

        print(f"   Procesada muestra {i+1}/{len(dataset_loader)}", end='\r')

    print("\n")

    # Convertir a DataFrames para análisis
    mr_df = pd.DataFrame(mr_stats)
    ct_df = pd.DataFrame(ct_stats)
    mask_df = pd.DataFrame(mask_stats)

    return mr_df, ct_df, mask_df

# Ejecutar análisis
mr_stats_df, ct_stats_df, mask_stats_df = analyze_intensity_statistics(eda_loader)

In [ ]:
# Mostrar estadísticas resumidas
print("\n" + "="*70)
print("ESTADÍSTICAS DE INTENSIDAD DEL DATASET COMPLETO")
print("="*70)

print("\nIMÁGENES MR:")
print(mr_stats_df.describe().round(2))

print("\nIMÁGENES CT:")
print(ct_stats_df.describe().round(2))

print("\nMÁSCARAS:")
print(mask_stats_df.describe().round(4))

# Información adicional sobre las máscaras
print(f"\nANÁLISIS DE MÁSCARAS:")
print(f"   Valores únicos promedio: {mask_stats_df['unique_values'].mean():.1f}")
print(f"   Ratio de voxels no-cero promedio: {mask_stats_df['non_zero_ratio'].mean():.3f}")
print(f"   Rango de valores únicos: {mask_stats_df['unique_values'].min()} - {mask_stats_df['unique_values'].max()}")

# Rangos de intensidad recomendados para normalización
print(f"\nRECOMENDACIONES PARA NORMALIZACIÓN:")
mr_p1, mr_p99 = np.percentile(mr_stats_df['min'].values), np.percentile(mr_stats_df['max'].values, 99)
ct_p1, ct_p99 = np.percentile(ct_stats_df['min'].values), np.percentile(ct_stats_df['max'].values, 99)

print(f"   MR: Rango recomendado [{mr_p1:.0f}, {mr_p99:.0f}]")
print(f"   CT: Rango recomendado [{ct_p1:.0f}, {ct_p99:.0f}]")
print(f"   Para CT típicamente se usa [-1000, 1000] HU (Hounsfield Units)")

In [ ]:
# Crear visualizaciones de las estadísticas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle(f'Distribución de Estadísticas de Intensidad - {DATASET_NAME}', fontsize=16, fontweight='bold')

# Primera fila: MR vs CT
axes[0, 0].boxplot([mr_stats_df['mean'], ct_stats_df['mean']], labels=['MR', 'CT'])
axes[0, 0].set_title('Intensidad Media')
axes[0, 0].set_ylabel('Valor de intensidad')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].boxplot([mr_stats_df['std'], ct_stats_df['std']], labels=['MR', 'CT'])
axes[0, 1].set_title('Desviación Estándar')
axes[0, 1].set_ylabel('Valor de intensidad')
axes[0, 1].grid(True, alpha=0.3)

# Histograma de rangos dinámicos
mr_range = mr_stats_df['max'] - mr_stats_df['min']
ct_range = ct_stats_df['max'] - ct_stats_df['min']
axes[0, 2].hist([mr_range, ct_range], bins=15, alpha=0.7, label=['MR', 'CT'], color=['blue', 'red'])
axes[0, 2].set_title('Rango Dinámico (Max - Min)')
axes[0, 2].set_xlabel('Rango de intensidad')
axes[0, 2].set_ylabel('Frecuencia')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Segunda fila: Análisis de máscaras y comparaciones
axes[1, 0].hist(mask_stats_df['unique_values'], bins=10, alpha=0.7, color='green', edgecolor='black')
axes[1, 0].set_title('Valores Únicos en Máscaras')
axes[1, 0].set_xlabel('Número de valores únicos')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(mask_stats_df['non_zero_ratio'], bins=15, alpha=0.7, color='orange', edgecolor='black')
axes[1, 1].set_title('Ratio de Voxels No-Cero en Máscaras')
axes[1, 1].set_xlabel('Ratio (0-1)')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].grid(True, alpha=0.3)

# Scatter plot: MR mean vs CT mean
axes[1, 2].scatter(mr_stats_df['mean'], ct_stats_df['mean'], alpha=0.6, color='purple')
axes[1, 2].set_xlabel('Intensidad Media MR')
axes[1, 2].set_ylabel('Intensidad Media CT')
axes[1, 2].set_title('Correlación MR vs CT (Intensidad Media)')
axes[1, 2].grid(True, alpha=0.3)

# Añadir línea de tendencia
coeff = np.polyfit(mr_stats_df['mean'], ct_stats_df['mean'], 1)
poly1d_fn = np.poly1d(coeff)
axes[1, 2].plot(mr_stats_df['mean'], poly1d_fn(mr_stats_df['mean']), '--k', alpha=0.8)

plt.tight_layout()
plt.show()

# Correlación entre MR y CT
correlation = np.corrcoef(mr_stats_df['mean'], ct_stats_df['mean'])[0, 1]
print(f"\nCorrelación entre intensidades medias MR-CT: {correlation:.3f}")

## 8. Resumen y Conclusiones

Resumen de los hallazgos principales del análisis exploratorio.

In [ ]:
print("\n" + "="*80)
print("RESUMEN DEL ANÁLISIS EXPLORATORIO")
print("="*80)

print(f"\nDATASET: {DATASET_NAME}")
print(f"   Número de muestras: {len(eda_dataset)}")
print(f"   Archivos analizados: {len(spacing_df)} volúmenes")

print(f"\nRESOLUCIÓN ESPACIAL (SPACING):")
print(f"   X: {spacing_df['image_spacing_x'].mean():.3f} ± {spacing_df['image_spacing_x'].std():.3f} mm")
print(f"   Y: {spacing_df['image_spacing_y'].mean():.3f} ± {spacing_df['image_spacing_y'].std():.3f} mm")
print(f"   Z: {spacing_df['image_spacing_z'].mean():.3f} ± {spacing_df['image_spacing_z'].std():.3f} mm")

print(f"\nDIMENSIONES (VOXELS):")
print(f"   X: {spacing_df['image_shape_x'].mean():.0f} ± {spacing_df['image_shape_x'].std():.0f}")
print(f"   Y: {spacing_df['image_shape_y'].mean():.0f} ± {spacing_df['image_shape_y'].std():.0f}")
print(f"   Z: {spacing_df['image_shape_z'].mean():.0f} ± {spacing_df['image_shape_z'].std():.0f}")

print(f"\nINTENSIDADES MR:")
print(f"   Rango: [{mr_stats_df['min'].mean():.0f}, {mr_stats_df['max'].mean():.0f}]")
print(f"   Media: {mr_stats_df['mean'].mean():.2f} ± {mr_stats_df['mean'].std():.2f}")

print(f"\nINTENSIDADES CT:")
print(f"   Rango: [{ct_stats_df['min'].mean():.0f}, {ct_stats_df['max'].mean():.0f}]")
print(f"   Media: {ct_stats_df['mean'].mean():.2f} ± {ct_stats_df['mean'].std():.2f}")

print(f"\nMÁSCARAS:")
print(f"   Valores únicos promedio: {mask_stats_df['unique_values'].mean():.1f}")
print(f"   Cobertura promedio: {mask_stats_df['non_zero_ratio'].mean()*100:.1f}% del volumen")

print(f"\nRECOMENDACIONES:")
print(f"   1. Usar spacing consistente para entrenamiento")
print(f"   2. Normalizar MR a [0, 1] o [-1, 1]")
print(f"   3. Normalizar CT usando rango [-1000, 1000] HU")
print(f"   4. Considerar augmentation espacial dado el spacing variable")
print(f"   5. Batch size = 1 es apropiado dadas las dimensiones variables")

print(f"\nAnálisis completado exitosamente")
print(f"\nPara cambiar el dataset, modifica la variable DATASET_PATH en la celda 4")
print(f"   Opciones disponibles: TASK1_HN, TASK1_AB, TASK1_TH, TASK2_HN, TASK2_AB, TASK2_TH")